# 국경일, 공휴일을 캐치해서 0,1로 분류하는 함수

In [287]:
import pandas as pd
import datetime

In [ ]:
dt = datetime.datetime.now()
dt

In [398]:
data = pd.read_excel('../2020데이터/01_제공데이터/performance_data.xlsx',header = 1)

In [399]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
방송일시     38309 non-null datetime64[ns]
노출(분)    21525 non-null float64
마더코드     38309 non-null int64
상품코드     38309 non-null int64
상품명      38309 non-null object
상품군      38309 non-null object
판매단가     38309 non-null int64
취급액      37372 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [400]:
data['방송일시'].dt.day.head()

0    1
1    1
2    1
3    1
4    1
Name: 방송일시, dtype: int64

# 공공데이터 open api

In [295]:
# 파라미터
import requests
from bs4 import BeautifulSoup

year = '2019'
mykey = 'H2U3MYpA5cdO9eidjraO87W8mSVkIo77SNPtdNmNm31T33EFszX9mnbji8AFKCZka6jDsAHG%2BCga0yEkZCim%2Fw%3D%3D'

holiday_2019 = []

for month in range(1,13):
    # int형으로 받은 숫자 중 10이하 한자리 수면 0을 붙여서 string으로 변환하고,
    # 아니면 그대로 string형으로 변환
    if month < 10:
        month = '0' + str(month)
    else:
        month = str(month)

    url = ' http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear=%s&solMonth=%s&ServiceKey=%s' % (year, month, mykey)
    
    get_data = requests.get(url)
    soup = BeautifulSoup(get_data.content, 'html.parser')
    table = soup.find_all('locdate')
    for i in table:
        holiday_2019.append(i.text)

In [298]:
holiday_2019

['20190101',
 '20190204',
 '20190205',
 '20190206',
 '20190301',
 '20190505',
 '20190506',
 '20190512',
 '20190606',
 '20190815',
 '20190912',
 '20190913',
 '20190914',
 '20191003',
 '20191009',
 '20191225']

# 휴일변수

In [401]:
holiday_2019 = pd.to_datetime(holiday_2019)

In [402]:
# (data['방송일시'].dt.date.isin(pd.to_datetime(holiday_2019)))
data['방송일시'].dt.date.isin(holiday_2019.date).head()

0    True
1    True
2    True
3    True
4    True
Name: 방송일시, dtype: bool

In [403]:
data['휴일변수'] = data['방송일시'].dt.weekday.isin([5,6]).values | (data['방송일시'].dt.date.isin(holiday_2019.date))

# 요일변수

# 요일변수

In [404]:
data['요일변수'] = data['방송일시'].dt.weekday

In [405]:
data['요일변수'].unique()

array([1, 2, 3, 4, 5, 6, 0], dtype=int64)

# 주말 변수

# 주말변수 생성(주말:토(5), 일(6))

In [408]:
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,휴일변수,요일변수,주말변수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,True,1,False
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,True,1,False
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,True,1,False
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,True,1,False
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,True,1,False


# 오전오후저녁 변수

In [409]:
data['오전오후저녁'] = data['방송일시']

In [410]:
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,휴일변수,요일변수,주말변수,오전오후저녁
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,True,1,False,2019-01-01 06:00:00
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,True,1,False,2019-01-01 06:00:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,True,1,False,2019-01-01 06:20:00
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,True,1,False,2019-01-01 06:20:00
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,True,1,False,2019-01-01 06:40:00


In [419]:
data['오전오후저녁'].loc[data['방송일시'].dt.hour.isin(['6','7','8','9','10','11','12'])] = 0
data['오전오후저녁'].loc[data['방송일시'].dt.hour.isin(['13','14','15','16','17','18'])] = 1
data['오전오후저녁'].loc[data['방송일시'].dt.hour.isin(['19','20','21','24','1','2'])] = 2

In [420]:
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,휴일변수,요일변수,주말변수,오전오후저녁
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,True,1,False,0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,True,1,False,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,True,1,False,0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,True,1,False,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,True,1,False,0


In [406]:
data['주말변수'] = data['요일변수'].isin([5,6]).values

# 휴일변수

In [212]:
data['방송일시'].dt.date[0]

datetime.date(2019, 1, 1)

In [213]:
date_holiday_2019.date[0]

datetime.date(2019, 1, 1)

In [214]:
(data['방송일시'].dt.date.isin(date_holiday_2019.date)).value_counts()

False    36482
True      1827
Name: 방송일시, dtype: int64

In [215]:
data['휴일변수'] = (data['요일변수'].isin([5,6])) | (data['방송일시'].dt.date.isin(date_holiday_2019.date))

In [167]:
date_holiday_2019 = pd.to_datetime(holiday_2019)

In [198]:
date_holiday_2019

DatetimeIndex(['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06',
               '2019-03-01', '2019-05-05', '2019-05-06', '2019-05-12',
               '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13',
               '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25'],
              dtype='datetime64[ns]', freq=None)

In [216]:
data[data['휴일변수']].head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,요일변수,주말변수,계절(반기별),휴일변수
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,False,False,True
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,False,False,True
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,False,False,True
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,False,False,True
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,False,False,True


In [174]:
len(date_holiday_2019)

16

In [175]:
data['방송일시'].isin(date_holiday_2019)

0    False
1    False
2    False
3    False
4    False
Name: 방송일시, dtype: bool

# 계절(분기별)

In [229]:
data['계절(분기별)'] = data['방송일시'].dt.quarter -1

In [230]:
data['계절(분기별)'].head()

0    0
1    0
2    0
3    0
4    0
Name: 계절(분기별), dtype: int64

# 오전, 오후, 저녁 변수

In [280]:
data['오전오후저녁'] = data['방송일시'].copy()

In [282]:
data['오전오후저녁'].unique()

array([0, Timestamp('2019-01-01 12:00:00'),
       Timestamp('2019-01-01 12:20:00'), ...,
       Timestamp('2020-01-01 01:00:00'), Timestamp('2020-01-01 01:20:00'),
       Timestamp('2020-01-01 01:40:00')], dtype=object)

In [277]:
del data['오전오후저녁']

In [407]:
data['주말변수'].describe()

count     38309
unique        2
top       False
freq      27025
Name: 주말변수, dtype: object

In [251]:
data['오전오후저녁'].head()

0   2019-01-01 06:00:00
1   2019-01-01 06:00:00
2   2019-01-01 06:20:00
3   2019-01-01 06:20:00
4   2019-01-01 06:40:00
Name: 오전오후저녁, dtype: datetime64[ns]

# 계절(반기별 변수)

In [147]:
s.head()

0    False
1    False
2    False
3    False
4    False
Name: 방송일시, dtype: bool

In [178]:
data['계절(반기별)'] = data['방송일시'].dt.month.isin([7,8,9,10,11,12])

# 계절(분기별)

In [184]:
data['계절(반기별)'].value_counts()

False    19191
True     19118
Name: 계절(반기별), dtype: int64